<h1> CORD-19 Solution Toolbox</h1>


We give here a minimal toolset to explore the dataset and start performing an EDA.  

<a id="0"></a>
We will provide the tools to:

* <a href='#1'>Browse through the files in the collections;</a>  
* <a href='#2'>Read content from JSON files;</a>  
* <a href='#3'>Bulk process JSON files to extract content in a DataFrame;</a>  
* <a href='#4'>Extract abstract content;</a>  
* <a href='#5'>Visualize text content;</a>  
* <a href='#6'>Visualize most frequent items in categorical features</a>  



<h1 style='background:#13E3E1; border:0; color:black'><center>Load packages</center></h1>

We just load the minimum packages for now.

In [ ]:
import numpy as np
import pandas as pd

import os
import json

<h1 style='background:#13E3E1; border:0; color:black'><center>Explore the data</center></h1>

In [ ]:
count = 0
file_exts = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        count += 1
        file_ext = filename.split(".")[-1]
        file_exts.append(file_ext)

file_ext_set = set(file_exts)

print(f"Files: {count}")
print(f"Files extensions: {file_ext_set}\n\n=====================\nFiles extension count:\n=====================")
file_ext_list = list(file_ext_set)
for fe in file_ext_list:
    fe_count = file_exts.count(fe)
    print(f"{fe}: {fe_count}")

Let's also look to the structure of directories, to see how the data is structured high-level:

In [ ]:
count = 0
for root, folders, filenames in os.walk('/kaggle/input'):
    print(root, folders)

Majority of files are in json format. The files are grouped in 3 main folders and multiple subfolders.


In [ ]:
print(f"pdf json: {len(os.listdir('/kaggle/input/CORD-19-research-challenge/document_parses/pdf_json'))}")
print(f"pmc json: {len(os.listdir('/kaggle/input/CORD-19-research-challenge/document_parses/pmc_json'))}")

<a href="#0"><small>Go to top</small></a>


We provide some tools to explore the jsons.

<a id="2"></a><h1 style='background:#13E3E1; border:0; color:black'><center>Read a JSON File</center></h1>



In [ ]:
json_folder_path = "/kaggle/input/CORD-19-research-challenge/document_parses/pdf_json"
json_file_name = os.listdir(json_folder_path)[0]
print(json_file_name)
json_path = os.path.join(json_folder_path, json_file_name)

with open(json_path) as json_file:
    json_data = json.load(json_file)

To use more easy, we can normalize the json. Here is the code.

In [ ]:
json_data_df = pd.io.json.json_normalize(json_data)

The json was transformed in a row in a dataframe, with the column names resulted by aggregating the succesive levels of the json structure.   Let's check the result.

In [ ]:
json_data_df

<a href="#0"><small>Go to top</small></a>

<a id="3"></a><h1 style='background:#13E3E1; border:0; color:black'><center>Convert the folder in a dataframe</center></h1>

Let's process now the folder. We will create a dataset with the data from the folder. We just take a subset of data (1000 samples).  For your work, just comment the line of code where the subset is declared and uncomment the line of code above, to process entire dataset.

In [ ]:
print(f"Files in folder: {len(os.listdir(json_folder_path))}")

In [ ]:
from tqdm import tqdm

# to process all files, uncomment the next line and comment the line below
# list_of_files = list(os.listdir(json_folder_path))
list_of_files = list(os.listdir(json_folder_path))[0:1000]
pmc_custom_license_df = pd.DataFrame()

for file in tqdm(list_of_files):
    json_path = os.path.join(json_folder_path, file)
    with open(json_path) as json_file:
        json_data = json.load(json_file)
    json_data_df = pd.io.json.json_normalize(json_data)
    pmc_custom_license_df = pmc_custom_license_df.append(json_data_df)

In [ ]:
pmc_custom_license_df.head()

<a href="#0"><small>Go to top</small></a>

<a id="4"></a><h1 style='background:#13E3E1; border:0; color:black'><center>Extract abstract text</center></h1>

Let's extract now abstract text from abstract column.  

Similar approach can be used to extract other parts from a dictionary-type field.


In [ ]:
pmc_custom_license_df['abstract_text'] = pmc_custom_license_df['abstract'].apply(lambda x: x[0]['text'] if x else "")

In [ ]:
pd.set_option('display.max_colwidth', 500)
pmc_custom_license_df[['abstract', 'abstract_text']].head()

<a href="#0"><small>Go to top</small></a>

<a id="5"></a><h1 style='background:#13E3E1; border:0; color:black'><center>Visualize text content</center></h1>

Let's present here few useful techniques for data visualization:

* Worldclouds for text fields;

* Countplot for category-type features.


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline 
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(15,15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=14)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
    
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=50,
        max_font_size=40, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(15,15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
    

In [ ]:
show_wordcloud(pmc_custom_license_df['abstract_text'], title = 'Comm use subset - papers abstract - frequent words  (500 samples)')

In [ ]:
show_wordcloud(pmc_custom_license_df['bib_entries.BIBREF0.title'], title = 'Comm use subset - papers title - frequent words (500 samples)')


<a href="#0"><small>Go to top</small></a>

<a id="6"></a><h1 style='background:#13E3E1; border:0; color:black'><center>Visualize most frequent items in categorical features</center></h1>


In [ ]:
pmc_custom_license_df.loc[((pmc_custom_license_df['bib_entries.BIBREF0.venue']=="") | ((pmc_custom_license_df['bib_entries.BIBREF0.venue'].isna()))), 'bib_entries.BIBREF0.venue'] = "Not identified"


In [ ]:
import seaborn as sns
def plot_count(feature, title, df, size=1, show_percents=False):
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    g = sns.countplot(df[feature], order = df[feature].value_counts().index[0:20], palette='Set3')
    g.set_title("Number of {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=10)
    if(show_percents):
        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x()+p.get_width()/2.,
                    height + 3,
                    '{:1.2f}%'.format(100*height/total),
                    ha="center") 
    ax.set_xticklabels(ax.get_xticklabels());
    plt.show()    


In [ ]:
plot_count('bib_entries.BIBREF0.venue', 'Comm use subset - Top 20 Journals (500 samples)', pmc_custom_license_df, 3.5)

We eliminate "Not identified"

In [ ]:
plot_count('bib_entries.BIBREF0.venue', 'Comm use subset - Top 20 Journals (500 samples)', 
           pmc_custom_license_df.loc[pmc_custom_license_df['bib_entries.BIBREF0.venue']!='Not identified'], 3.5)

<a href="#0"><small>Go to top</small></a>